# ch4 函數與模型

使用節點可以組織出複雜的運算系統，但節點也只是起了連接網路內各個元件的功能而已。支撐起整套運算系統的核心是模型和函數，下面我們就來簡單介紹一下兩者的差別:

**`函數:`**
1. 定義在unit_function模組中，功能是提供一些常見的函數，比如:tanh或是cross_entropy。
2. 使用時不需要事先導入到網路，直接在建立連接時使用即可。
3. 函數沒有需要優化的參數所以屬於資料節點。

**`模型:`**
1. 定義在layer模組中，功能是提供一些常見的(單層)類神經網路模型，比如:CNN或LSTM
2. 使用時需要導入到網路，否則運行節點時會保錯。
3. 模型需要優化的參數所以屬於模型節點。

In [112]:
from nyto import unit_function as uf
from nyto import layer
from nyto import net_tool as to

## 函數

函數有嚴格的使用規定，主要分成支援右移運算和不支援右移運算兩種，支援右移運算的可以寫成下面兩種形式:

    [1] node1 >> function(param,) >> node2
    [2] node2 = function(param,)(node1)

不支援右移運算只能寫成下面這種形式:
    
    [1] function(node1, param,) >> node2
    [2] node2 = function(node1, param,)
    
通常不支援右移運算的函數都是需要輸入兩個或兩個節點以上的函數，所以無法支援右移運算。

**`支援右移運算:`**
* linear()
* relu()
* gaussian()
* sigmoid()
* tanh()
* col_nor()
* row_nor()
* softmax()
* global_average_pooling()
* global_max_pooling()
* flattening()
* max_pooling(kernel_shape_node_if, strides=1)
* average_pooling(kernel_shape_node_if, strides=1)

**`不支援右移運算:`**
* concatenate(*node_ifs, axis=1)
* tile(node_if, size_tuple)
* MSE(pre_if, target_if)
* RMSE(pre_if, target_if)
* MAE(pre_if, target_if)
* MAPE(pre_if, target_if)
* cross_entropy(pre_if, target_if)
* binary_cross_entropy(pre_if, target_if)
* accuracy(pre_if, target_if)

由於大部份的函數使用上都算直覺，這邊就挑幾個說明。

### max_pooling和average_pooling

這兩個函數是卷集神經網路會用到的pooling函數，需要調整的參數為:
* (tuple/node_interface)kernel_shape_node_if: 窗口的尺寸，比如(3,3)，也可以輸入一個節點界面。
* (int/node_interface)strides=1: 窗口移動的大小

In [113]:
import numpy as np

nn,n = to.new_net(
    # 輸入的圖片是4維的numpy.array
    # [第幾筆資料][channel][圖片row][圖片col]
    img_np = to.add_data(np.arange(2*3*5*5).reshape(2,3,5,5)),
    kernal_shape=(3,3),
    strides=1
)

n.img_np >> uf.max_pooling(n.kernal_shape, n.strides) >> n.max_pooling
n.img_np >> uf.average_pooling((3,3), strides=1) >> n.average_pooling

### concatenate和tile

這兩個函數是用來拼接numpy.array用的可以對應到numpy的函數:
* concatenate(*node_ifs, axis=1) -> np.concatenate(node_ifs, axis=axis)
* tile(node_if, size_tuple) -> np.tile(node_if, size_tuple)

In [114]:
nn,n = to.new_net(
    data1 = np.arange(9).reshape(3,3),
    data2 = np.arange(9,18).reshape(3,3),
    size_tuple = (3,2)
)

n.data3 = uf.concatenate(n.data1, n.data2)
n.data4 = uf.tile(n.data1, size_tuple=n.size_tuple)

In [115]:
to.get(n.data3)

array([[ 0,  1,  2,  9, 10, 11],
       [ 3,  4,  5, 12, 13, 14],
       [ 6,  7,  8, 15, 16, 17]])

In [116]:
to.get(n.data4)

array([[0, 1, 2, 0, 1, 2],
       [3, 4, 5, 3, 4, 5],
       [6, 7, 8, 6, 7, 8],
       [0, 1, 2, 0, 1, 2],
       [3, 4, 5, 3, 4, 5],
       [6, 7, 8, 6, 7, 8],
       [0, 1, 2, 0, 1, 2],
       [3, 4, 5, 3, 4, 5],
       [6, 7, 8, 6, 7, 8]])

## 模型

下面來介紹模型，這邊需要注意的是前面我們介紹過模型節點，所謂的模型節點就是裡面保存了模型的節點。而模型節點最大的特點就是模型是可以被優化的，不能被優化的我們一般會被放到資料節點中。

這邊我們要做更近一步的說明了，首先:可以被優化的單元一定是模型，但模型不一定可以被優化。也就是說模型中其實也存在可以優化的模型與不能優化的模型。你可能會感到困惑，那資料與模型的差別在於什麼呢？

模型與資料的真正定義是:
> 可以被多個網路共享的單元為資料，各網路獨有的單元為模型

這說明模型與資料的差別並不是簡單的能不能優化而已，而在模型中存在著可以被優化與不能被優化的模型，我們將可以優化的模型稱為層模型(layer)。層模型被定義在`layer`模組中，有以下幾個成員:
1. variable_layer: 單純的矩陣，一般用於尋找最佳的輸入
2. nn_layer:       單層的類神經網路
3. lstm_layer:     單層的LSTM
4. conv_layer:     單層卷集層，裡面有多個keranl

### variable_layer

作為一個可優化的矩陣，能做到的事情遠比看上去的多。基本上，另外3種的層模型都可以使用該層去做出來，只不過會犧牲執行的效率。可以使用*layer.new_variable_layer*函數產生，下面是各項參數的說明:

**`layer.new_variable_layer`**

**structure**
    
    生成矩陣的形狀，資料型別是tuple。
    exp:(3,2)
**init_values**
    
    亂數生成時常態分配的平均數，預設0。
**random_size**
    
    亂數生成時常態分配的標準差,為None時則使用init_values作為預設值。
    預設None。
**dropout**

    使用dropout訓練時是否啟用dropout，預設False。

In [117]:
variable_mod = layer.new_variable_layer((3,4))
variable_mod

variable_layer((3, 4))

可以使用`shape`方法查看模型的形狀:

In [118]:
variable_mod.shape

(3, 4)

可以使用`values`方法查看並修改模型參數:

In [119]:
variable_mod.values

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

如果你手邊有一個已經準備好的`numpy.array`，你可以使用*layer.np_to_variable_layer*轉換成`variable_layer`，下面是該函數的參數:

**`layer.np_to_variable_layer`**

**variable_np**
    
    要被轉換的numpy.array。    
**dropout**

    使用dropout訓練時是否啟用dropout，預設False。

In [120]:
variable_mod=layer.np_to_variable_layer(variable_np=np.arange(12).reshape(3,4))
variable_mod

variable_layer((3, 4))

In [121]:
variable_mod.values

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

需要注意的是`variable_layer`在網路中被執行時，如果需要與其他的不是層模型的模型互動時，建議先轉換成np.array。

與其他層模型互動時，不需要傳換:

In [122]:
nn,n = to.new_net(
    var1_layer=layer.new_variable_layer((3,3), 1), 
    var2_layer=layer.new_variable_layer((3,3), 2),
    nn_layer=layer.new_nn_layer((3,3)),
    data_np=np.arange(9).reshape(3,3)
)

n.interact_with_other_layer1 = n.var1_layer + n.var2_layer
n.interact_with_other_layer2 = n.var1_layer >> n.nn_layer

to.get(n.interact_with_other_layer1, n.interact_with_other_layer2)

(variable_layer((3, 3)),
 array([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]))

與非層模型互動時，可以先使用`values`方法取出模型中的參數:

In [123]:
n.use_values_to_get_np = n.var1_layer.values() + n.data_np
to.get(n.use_values_to_get_np)

array([[1., 2., 3.],
       [4., 5., 6.],
       [7., 8., 9.]])

### nn_layer

單層的神經網路，可以用來組合出複雜的網路結構，可以使用*layer.new_nn_layer*產生:

**`layer.new_nn_layer`**

**structure**

    網路結構，資料型別是tuple。
    第一格是網路的輸入大小，第一格是網路的輸出大小。
    exp:(3,2)  
**init_values**

    亂數生成時常態分配的平均數，資料型別是tuple，預設為(0,0)。
    第一格是網路權重的平均數，第二格是網路偏置的平均數。
**random_size**

    亂數生成時常態分配的標準差,為None時則使用init_values作為預設值。
    第一格是網路權重的標準差，第二格是網路偏置的標準差。
    預設為(None, None)。
**dropout**
    
    使用dropout訓練時是否啟用dropout，預設True。

In [124]:
nn_mod = layer.new_nn_layer((2,3))
nn_mod

nn_layer((2, 3))

可以使用`weights`和`bias`方法取得並修改權重與偏置的參數:

In [125]:
nn_mod.weights

array([[0., 0., 0.],
       [0., 0., 0.]])

In [126]:
nn_mod.bias

array([[0., 0., 0.]])

可以使用`__call__`方法來對輸入的np.array做運算:

In [127]:
data_np = np.arange(6).reshape(3,2)
nn_mod(data_np)

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

### lstm_layer

單層的神經網路，可以用來對時間序列分析，可以使用*layer.new_lstm_layer*產生:

**`layer.new_lstm_layer`**

**structure**

    網路結構，資料型別是tuple。
    第一格是網路的輸入大小，第二格是網路的輸出大小。
    exp:(3,2)  
**compute_init**

    compute計算單元網路參數亂數生成時常態分配的平均數，資料型別是tuple，預設為(0,0)。
    第一格是網路權重的平均數，第二格是網路偏置的平均數。
**compute_random**

    compute計算單元網路參數亂數生成時常態分配的標準差,為None時則使用init_values作為預設值。
    第一格是網路權重的標準差，第二格是網路偏置的標準差。
    預設為(None, None)。
**input_init**

    input gate計算單元網路參數亂數生成時常態分配的平均數，資料型別是tuple，預設為(0,0)。
    第一格是網路權重的平均數，第二格是網路偏置的平均數。
**input_random**

    input gate計算單元網路參數亂數生成時常態分配的標準差,為None時則使用init_values作為預設值。
    第一格是網路權重的標準差，第二格是網路偏置的標準差。
    預設為(None, None)。
**output_init**

    output gate計算單元網路參數亂數生成時常態分配的平均數，資料型別是tuple，預設為(0,0)。
    第一格是網路權重的平均數，第二格是網路偏置的平均數。
**output_random**

    output gate計算單元網路參數亂數生成時常態分配的標準差,為None時則使用init_values作為預設值。
    第一格是網路權重的標準差，第二格是網路偏置的標準差。
    預設為(None, None)。
**forget_init**

    forget gate計算單元網路參數亂數生成時常態分配的平均數，資料型別是tuple，預設為(0,0)。
    第一格是網路權重的平均數，第二格是網路偏置的平均數。
**forget_random**

    forget gate計算單元網路參數亂數生成時常態分配的標準差,為None時則使用init_values作為預設值。
    第一格是網路權重的標準差，第二格是網路偏置的標準差。
    預設為(None, None)。
**state_init**

    預設初始狀態(memory cell)參數亂數生成時常態分配的平均數，預設為0。
**state_random**

    預設初始狀態(memory cell)參數亂數生成時常態分配的標準差,為None時則使用init_values作為預設值。
    預設為None。
**input_data_dropout**
    
    連接輸入feature的權重是否啟用dropout，預設True。
**mem_data_dropout**
    
    連接輸入memory cell的權重是否啟用dropout，預設False。
**reinput_data_dropout**
    
    連接輸入上個時間點書出的權重是否啟用dropout，預設True。
**com_func**
    
    compute計算單元的activation function，預設為tanh
**ingate_func**
    
    input gate計算單元的activation function，預設為sigmoid
**outgate_func**
    
    output gate計算單元的activation function，預設為sigmoid
**forgate_func**
    
    forget gate計算單元的activation function，預設為sigmoid
**final_func**
    
    lstm模型輸出時的activation function，預設為tanh

In [128]:
lstm_mod = layer.new_lstm_layer((2,1))
lstm_mod

lstm_layer((2, 1))

下面是常用的模型參數與方法:
1. shape: 模型的形狀(輸入大小,輸出大小)
2. init_mem: 預設初始memory call參數(variable_layer)
3. init_out: 預設初始reinput_data參數(variable_layer)

In [129]:
lstm_mod.init_mem

variable_layer((1, 1))

In [130]:
lstm_mod.init_out

variable_layer((1, 1))

**`lstm運算`**

lstm的運算根據輸入資料的情況分成3種模式:
1. 單筆(run): 只計算某個時間點，不僅需要輸入特徵資料，還需要memory cell與reinput_data。
2. 多筆(run_series): 計算時間序列，只需要輸入特徵資料，memory cell與reinput_data會自動計算。
3. 批(__call__): 計算批資料。

下面是計算單筆的範例，計算結果會回傳lstm的結果和新的memory:

In [131]:
feature_np = np.array([[1,0]])
memory_np = lstm_mod.init_mem.values
reinput_data = lstm_mod.init_out.values

final_np, new_mem = lstm_mod.run(in_np=feature_np, mem_np=memory_np, out_np=reinput_data)
final_np, new_mem

(array([[0.]]), array([[0.]]))

下面是計算多筆的範例，計算結果會回傳lstm的各時間點的結果:

In [132]:
feature_np = np.arange(10).reshape(5,2)
lstm_mod.run_series(feature_np)

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.]])

下面是計算批的範例，計算結果會回傳lstm的各批在各時間點的結果:

In [133]:
batch1 = np.arange(6).reshape(3,2)
batch2 = np.arange(10).reshape(5,2)
total_batch = [batch1, batch2]

lstm_mod(total_batch)

[array([[0.],
        [0.],
        [0.]]),
 array([[0.],
        [0.],
        [0.],
        [0.],
        [0.]])]

### conv_layer

單層的卷集層，可以用來對圖像資料分析，可以使用*layer.new_conv_layer*產生:

**`layer.new_conv_layer`**

**structure**

    filter結構，資料型別是tuple。
    第一格是input大小，第二格是output大小。
    exp:(1,2)
**kernal_size**
    
    kernel結構，資料型別是tuple，預設(3,3)。
    第二格是kernel的row大小,第三格是kernel的col大小。
**init_values**

    亂數生成時常態分配的平均數，預設0。
**random_size**

    亂數生成時常態分配的標準差,為None時則使用init_values作為預設值。
    預設None。
**dropout**
    
    使用dropout訓練時是否啟用dropout，資料型別是bool。
    預設False。
    
**pad_mod**

    padding模式，有三種選擇:(1)'full'(2)'valid'(3)'same'
    'same'=不做padding，預設為'valid'
    
**strides**

    窗口移動大小，資料型別是int。
    預設為1。

In [134]:
cnn=layer.new_conv_layer((1,2))
cnn

conv_layer(((1, 2), (3, 3)))

可以使用`shape`查看模型的形狀:

In [135]:
filter_shape, kernal_shape = cnn.shape
filter_shape, kernal_shape

((1, 2), (3, 3))

可以使用`filter_np`查看或修改filter的參數:

In [136]:
cnn.filter_np

array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]],


       [[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]])

`CNN計算`

輸入到conv_layer的圖片資料必須是一個4維的np.array的資料格式，由外到內分別是:
1. 第幾筆資料
2. 圖片的channel(feature)
3. 圖片的row
4. 圖片的column

In [137]:
# img_np[2, 1, 3, 3]
# 2筆資料
# 1個channel
# row = 3
# column = 3
img_np = np.array([
    [
        [
            [0, 1, 0],
            [0, 1, 0],
            [0, 1, 0],
        ]
    ],
    [
        [
            [0, 0, 0],
            [1, 1, 1],
            [0, 0, 0],
        ]
    ]
])

cnn會將圖片的一個channal，提取出2個feature

In [138]:
ret_np = cnn(img_np)
ret_np.shape

(2, 2, 3, 3)

使用不同的pad_mod，所得到的輸出尺寸將不同:

In [139]:
same_pad_cnn = layer.new_conv_layer((1,1), pad_mod='same')
valid_pad_cnn = layer.new_conv_layer((1,1), pad_mod='valid')
full_pad_cnn = layer.new_conv_layer((1,1), pad_mod='full')

same_pad_cnn(img_np).shape, valid_pad_cnn(img_np).shape, full_pad_cnn(img_np).shape

((2, 1, 1, 1), (2, 1, 3, 3), (2, 1, 5, 5))

## 子網路

如果你手邊正好有一個訓練好的網路，它能不能與其他網路組合起來變成一個更好的網路呢？被其他網路導入的網路我們稱之為子網路，我們可以使用`net.push_get`來在其他網路中完成對該網路的使用:

In [140]:
# 子網路
sub_nn, sub_nn_node = to.new_net()
sub_nn_node.y = sub_nn_node.x1 + sub_nn_node.x2

# 導入子網路
nn, node = to.new_net(a=1, b=2, sub_nn=sub_nn)
node.sub_nn_return = node.sub_nn.push_get(
    {'y'}, x1=node.a, x2=node.b
)

to.get(node.sub_nn_return)

{'y': 3}

對於子網路，我們應該要如何看待呢？可以簡單當成層模型來使用。當優化時，被導入的子網路也會如同網路內的其他層模型一樣被優化。但如果只想使用而不想優化子網路，最簡單的方法就是在導入時使用資料的方式導入。當然更簡單的方式也是有的，使用者可以很方便的切換是否需要優化子網路，我們在下一章中再做介紹。

***

*END*